# Hosting Strands + LangGraph agent with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. 

We will focus on a LangGraph with Amazon Bedrock model example. For Strands Agents with Amazon Bedrock model check [here](../01-strands-with-bedrock-model)
and for a Strands Agents with an OpenAI model check [here](../03-strands-with-openai-model).

### Tutorial Details

| Information         | Details                                                                      |
|:--------------------|:-----------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                               |
| Agent type          | Single                                                                       |
| Agentic Framework   | Strands + LangGraph                                                                    |
| LLM model           | Anthropic Claude Sonnet 3                                                    |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands + LangGraph and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                               |
| Example complexity  | Easy                                                                         |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                 |

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime. 

For demonstration purposes, we will  use a LangGraph agent using Amazon Bedrock models

In our example we will use a very simple agent with two tools: `get_weather` and `get_time`. 

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Bedrock models
* Using LangGraph


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* LangGraph
* Docker running

## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

The architecture here will look as following:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="60%"/>
</div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../../.."
sys.path.append(os.path.abspath(module_path))

In [3]:
from textwrap import dedent
from graph import build_graph
from src.utils.common_utils import get_message_from_string
from src.utils.strands_sdk_utils import strands_utils
from src.prompts.template import apply_prompt_template

In [4]:
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Default level is INFO
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)

def enable_debug_logging():
    """Enable debug level logging for more detailed execution information."""
    logging.getLogger(__name__).setLevel(logging.DEBUG)

# 로거 설정을 전역으로 한 번만 수행
logger = logging.getLogger(__name__)
logger.propagate = False
for handler in logger.handlers[:]:
    logger.removeHandler(handler)
handler = logging.StreamHandler()
formatter = logging.Formatter('\n%(levelname)s [%(name)s] %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)  # 기본 레벨은 INFO로 설정

class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'


In [5]:
# Create the graph
graph = build_graph()

In [6]:
def run_agent_workflow(user_input: str, debug: bool = False):
    """Run the agent workflow with the given user input.

    Args:
        user_input: The user's query or request
        debug: If True, enables debug level logging

    Returns:
        The final state after the workflow completes
    """
    if not user_input:
        raise ValueError("Input could not be empty")

    if debug:
        enable_debug_logging()

    #logger.info(f"Starting workflow with user input: {user_input}")
    logger.info(f"{Colors.GREEN}===== Starting workflow ====={Colors.END}")
    logger.info(f"{Colors.GREEN}\nuser input: {user_input}{Colors.END}")
    
    user_prompts = dedent(
        '''
        Here is a user request: <user_request>{user_request}</user_request>
        '''
    )
    context = {"user_request": user_input}
    user_prompts = user_prompts.format(**context)
    messages = [get_message_from_string(role="user", string=user_prompts, imgs=[])]

        
    result = graph.invoke(
        input={
            # Runtime Variables
            "messages": messages,
            "request": user_input,
            "request_prompt": user_prompts
        },
        config={
            "recursion_limit": 100
        }
    )
    logger.debug(f"{Colors.RED}Final workflow state: {result}{Colors.END}")
    logger.info(f"{Colors.GREEN}===== Workflow completed successfully ====={Colors.END}")
    return result

In [7]:
import nest_asyncio
nest_asyncio.apply()

#remove_artifact_folder()

user_query = '''
    안녕 나는 장동진이라고 해. 만나서 반가워
    나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어. 
'''

result = run_agent_workflow(
    user_input=user_query,
    debug=False
)



INFO [__main__] ===== Starting workflow =====

INFO [__main__] 
user input: 
    안녕 나는 장동진이라고 해. 만나서 반가워
    나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어. 


INFO [graph.nodes] ===== Coordinator talking...... =====

INFO [src.utils.strands_sdk_utils] COORDINATOR - Prompt Cache Disabled


안녕하세요 장동진님! 저는 Bedrock-Manus입니다. 만나서 반갑습니다!

데이터 분석과 인사이트 추출에 대한 요청은 저희 플래너 시스템과 상담이 필요할 것 같습니다.

handoff_to_planner: I'll need to consult our planning system for this request.


INFO [graph.nodes] ===== Coordinator completed task =====

INFO [graph.nodes] ===== Planner generating full plan =====

INFO [src.utils.strands_sdk_utils] PLANNER - Prompt Cache Disabled


KeyboardInterrupt: 

In [8]:
%%writefile strands_langgraph_bedrock.py

import sys, os
import json
import logging
import argparse
from textwrap import dedent
from graph import build_graph
from src.utils.common_utils import get_message_from_string
from src.utils.strands_sdk_utils import strands_utils
from src.prompts.template import apply_prompt_template

from bedrock_agentcore.runtime import BedrockAgentCoreApp

# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Default level is INFO
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)

def enable_debug_logging():
    """Enable debug level logging for more detailed execution information."""
    logging.getLogger(__name__).setLevel(logging.DEBUG)

# 로거 설정을 전역으로 한 번만 수행
logger = logging.getLogger(__name__)
logger.propagate = False
for handler in logger.handlers[:]:
    logger.removeHandler(handler)
handler = logging.StreamHandler()
formatter = logging.Formatter('\n%(levelname)s [%(name)s] %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)  # 기본 레벨은 INFO로 설정

class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'


# Create the graph
graph = build_graph()

def run_agent_workflow(user_input: str, debug: bool = False):
    """Run the agent workflow with the given user input.

    Args:
        user_input: The user's query or request
        debug: If True, enables debug level logging

    Returns:
        The final state after the workflow completes
    """
    if not user_input:
        raise ValueError("Input could not be empty")

    if debug:
        enable_debug_logging()

    #logger.info(f"Starting workflow with user input: {user_input}")
    logger.info(f"{Colors.GREEN}===== Starting workflow ====={Colors.END}")
    logger.info(f"{Colors.GREEN}\nuser input: {user_input}{Colors.END}")
    
    user_prompts = dedent(
        '''
        Here is a user request: <user_request>{user_request}</user_request>
        '''
    )
    context = {"user_request": user_input}
    user_prompts = user_prompts.format(**context)
    messages = [get_message_from_string(role="user", string=user_prompts, imgs=[])]

        
    result = graph.invoke(
        input={
            # Runtime Variables
            "messages": messages,
            "request": user_input,
            "request_prompt": user_prompts
        },
        config={
            "recursion_limit": 100
        }
    )
    logger.debug(f"{Colors.RED}Final workflow state: {result}{Colors.END}")
    logger.info(f"{Colors.GREEN}===== Workflow completed successfully ====={Colors.END}")
    return result

def strands_langgraph_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_query = payload.get("prompt")

    result = run_agent_workflow(
        user_input=user_query,
        debug=False
    )
    
    return result

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_langgraph_bedrock(json.loads(args.payload))
    print(response)



Overwriting strands_langgraph_bedrock.py


#### Invoking local agent

In [9]:
!python strands_langgraph_bedrock.py '{"prompt": "안녕 나는 장동진이라고 해. 만나서 반가워 나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어."}'


INFO [__main__] ===== Starting workflow =====

INFO [__main__] 
user input: 안녕 나는 장동진이라고 해. 만나서 반가워 나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어.

INFO [graph.nodes] ===== Coordinator talking...... =====

INFO [src.utils.strands_sdk_utils] COORDINATOR - Prompt Cache Disabled
안녕하세요 장동진님! 저는 Bedrock-Manus입니다. 만나서 반갑습니다!

데이터 분석과 인사이트 추출에 대한 요청은 저희 플래너 시스템과 상담이 필요할 것 같습니다.

handoff_to_planner: I'll need to consult our planning system for this request.
INFO [graph.nodes] ===== Coordinator completed task =====

INFO [graph.nodes] ===== Planner generating full plan =====

INFO [src.utils.strands_sdk_utils] PLANNER - Prompt Cache Disabled
# Plan

## thought
- 사용자가 자신을 소개하고 데이터 분석에 대한 일반적인 관심을 표현했습니다.
- 구체적인 데이터나 분석 요구사항이 제시되지 않았으므로, 먼저 기본적인 소개와 가능한 분석 방향을 제시하는 것이 좋겠습니다.
- 데이터 분석 프로세스와 가능한 인사이트 도출 방법에 대해 안내가 필요합니다.

## title
데이터 분석 및 인사이트 도출 프로세스 안내

## steps
### 1. Reporter: 초기 응답 및 프로세스 안내
- [ ] 사용자 환영 인사 작성
- [ ] 데이터 분석 프로세스 개요 설명
- [ ] 필요한 정보 요청사항 작성
  - 분석하고자 하는 데이터의 종류
  - 데이터의 형식 (CSV, Excel 등)


## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### Strands Agent SDK + LangGraph with Amazon Bedrock model
Let's start with our Strands Agent SDK + LangGraph using Amazon Bedrock model. Other examples with different frameworks and models are available in the parent directories

### Dockerfile


In [10]:
%%writefile strands_langgraph_bedrock.py

import json
import logging
import argparse
from textwrap import dedent
from graph import build_graph
from src.utils.common_utils import get_message_from_string
from src.utils.strands_sdk_utils import strands_utils
from src.prompts.template import apply_prompt_template
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Default level is INFO
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)

def enable_debug_logging():
    """Enable debug level logging for more detailed execution information."""
    logging.getLogger(__name__).setLevel(logging.DEBUG)

# 로거 설정을 전역으로 한 번만 수행
logger = logging.getLogger(__name__)
logger.propagate = False
for handler in logger.handlers[:]:
    logger.removeHandler(handler)
handler = logging.StreamHandler()
formatter = logging.Formatter('\n%(levelname)s [%(name)s] %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)  # 기본 레벨은 INFO로 설정

class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'


# Create the graph
graph = build_graph()

def run_agent_workflow(user_input: str, debug: bool = False):
    """Run the agent workflow with the given user input.

    Args:
        user_input: The user's query or request
        debug: If True, enables debug level logging

    Returns:
        The final state after the workflow completes
    """
    if not user_input:
        raise ValueError("Input could not be empty")

    if debug:
        enable_debug_logging()

    #logger.info(f"Starting workflow with user input: {user_input}")
    logger.info(f"{Colors.GREEN}===== Starting workflow ====={Colors.END}")
    logger.info(f"{Colors.GREEN}\nuser input: {user_input}{Colors.END}")
    
    user_prompts = dedent(
        '''
        Here is a user request: <user_request>{user_request}</user_request>
        '''
    )
    context = {"user_request": user_input}
    user_prompts = user_prompts.format(**context)
    messages = [get_message_from_string(role="user", string=user_prompts, imgs=[])]

        
    result = graph.invoke(
        input={
            # Runtime Variables
            "messages": messages,
            "request": user_input,
            "request_prompt": user_prompts
        },
        config={
            "recursion_limit": 100
        }
    )
    logger.debug(f"{Colors.RED}Final workflow state: {result}{Colors.END}")
    logger.info(f"{Colors.GREEN}===== Workflow completed successfully ====={Colors.END}")
    return result

@app.entrypoint
def strands_langgraph_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_query = payload.get("prompt")

    result = run_agent_workflow(
        user_input=user_query,
        debug=False
    )

    return result

if __name__ == "__main__":
    
    app.run()


Overwriting strands_langgraph_bedrock.py


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Creating runtime role

Before starting, let's create an IAM role for our AgentCore Runtime. We will do so using the utils function pre-developed for you.

In [11]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.join(utils_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import create_agentcore_role

agent_name="strands_langgraph_agentcore"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

sys.path[0]: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials
Role already exists -- deleting and creating it again
policies: {'PolicyNames': ['AgentCorePolicy'], 'IsTruncated': False, 'ResponseMetadata': {'RequestId': '19264be3-a477-47ff-90da-a0f9df4900a8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 06 Aug 2025 04:22:14 GMT', 'x-amzn-requestid': '19264be3-a477-47ff-90da-a0f9df4900a8', 'content-type': 'text/xml', 'content-length': '380'}, 'RetryAttempts': 0}}
deleting agentcore-strands_langgraph_agentcore-role
recreating agentcore-strands_langgraph_agentcore-role
attaching role policy agentcore-strands_langgraph_agentcore-role


### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

**Signature:**
```python
agentcore_runtime.configure(
    entrypoint: str,
    execution_role: Optional[str] = None,
    agent_name: Optional[str] = None,
    requirements: Optional[List[str]] = None,
    requirements_file: Optional[str] = None,
    ecr_repository: Optional[str] = None,
    container_runtime: Optional[str] = None,
    auto_create_ecr: bool = True,
    auto_create_execution_role: bool = False,
    authorizer_configuration: Optional[Dict[str, Any]] = None,
    region: Optional[str] = None,
    protocol: Optional[Literal['HTTP', 'MCP']] = None,
) -> bedrock_agentcore_starter_toolkit.operations.runtime.models.ConfigureResult
```

**Docstring:**  
Configure Bedrock AgentCore from notebook using an entrypoint file.

**Args:**
- **entrypoint**: Path to Python file with optional Bedrock AgentCore name (e.g., "handler.py" or "handler.py:bedrock_agentcore")
- **execution_role**: AWS IAM execution role ARN or name (optional if auto_create_execution_role=True)
- **agent_name**: name of the agent
- **requirements**: Optional list of requirements to generate requirements.txt
- **requirements_file**: Optional path to existing requirements file
- **ecr_repository**: Optional ECR repository URI
- **container_runtime**: Optional container runtime (docker/podman)
- **auto_create_ecr**: Whether to auto-create ECR repository
- **auto_create_execution_role**: Whether to auto-create execution role (makes execution_role optional)
- **authorizer_configuration**: JWT authorizer configuration dictionary
- **region**: AWS region for deployment
- **protocol**: agent server protocol, must be either HTTP or MCP

In [12]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    agent_name=agent_name,
    entrypoint="strands_langgraph_bedrock.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region
)
response

Entrypoint parsed: file=/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/strands_langgraph_bedrock.py, bedrock_agentcore_name=strands_langgraph_bedrock
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.entrypoint:Entrypoint parsed: file=/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/strands_langgraph_bedrock.py, bedrock_agentcore_name=strands_langgraph_bedrock
Configuring BedrockAgentCore agent: strands_langgraph_agentcore
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Configuring BedrockAgentCore agent: strands_langgraph_agentcore
Generated Dockerfile: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/Dockerfile
INFO:b

ConfigureResult(config_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/Dockerfile'), dockerignore_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/.dockerignore'), runtime='Docker', region='us-west-2', account_id='615299776985', execution_role='arn:aws:iam::615299776985:role/agentcore-strands_langgraph_agentcore-role', ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [13]:
launch_result = agentcore_runtime.launch()

Launching Bedrock AgentCore agent 'strands_langgraph_agentcore' to cloud
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Launching Bedrock AgentCore agent 'strands_langgraph_agentcore' to cloud


Build: Successfully built 0d4c1235539b
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.container:Build: Successfully built 0d4c1235539b
Build: Successfully tagged bedrock_agentcore-strands_langgraph_agentcore:latest
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.container:Build: Successfully tagged bedrock_agentcore-strands_langgraph_agentcore:latest
Docker image built: bedrock_agentcore-strands_langgraph_agentcore:latest
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Docker image built: bedrock_agentcore-strands_langgraph_agentcore:latest
Using execution role from config: arn:aws:iam::615299776985:role/agentcore-strands_langgraph_agentcore-role
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Using execution role from config: arn:aws:iam::615299776985:role/agentcore-strands_langgraph_agentcore-role
✅ Execution role validation passed: arn:aws:iam::615299776985:role/agentcore-strands_langgraph_agentcore-role
INFO:bedrock_agentcore_starter_too

✅ Reusing existing ECR repository: 615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_langgraph_agentcore


Registry authentication successful
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.container:Registry authentication successful
Tagging image: bedrock_agentcore-strands_langgraph_agentcore:latest -> 615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_langgraph_agentcore:latest
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.container:Tagging image: bedrock_agentcore-strands_langgraph_agentcore:latest -> 615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_langgraph_agentcore:latest
Pushing image to registry...
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.container:Pushing image to registry...


The push refers to repository [615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_langgraph_agentcore]
89db9549416c: Preparing
8069433ef2a8: Preparing
630c4be75fed: Preparing
5a2b9050aa4c: Preparing
6ed6547a37cd: Preparing
2e763e579d90: Preparing
05ed7b2bcaa5: Preparing
d74514903961: Preparing
e26286284f6e: Preparing
dd97e58b4e81: Preparing
d74514903961: Waiting
dd97e58b4e81: Waiting
2e763e579d90: Waiting
05ed7b2bcaa5: Waiting
e26286284f6e: Waiting
630c4be75fed: Layer already exists
8069433ef2a8: Layer already exists
6ed6547a37cd: Layer already exists
5a2b9050aa4c: Layer already exists
2e763e579d90: Layer already exists
05ed7b2bcaa5: Layer already exists
d74514903961: Layer already exists
e26286284f6e: Layer already exists
dd97e58b4e81: Layer already exists
89db9549416c: Pushed


Image pushed successfully
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.container:Image pushed successfully
Image uploaded to ECR: 615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_langgraph_agentcore
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Image uploaded to ECR: 615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_langgraph_agentcore
Deploying to Bedrock AgentCore...
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Deploying to Bedrock AgentCore...


latest: digest: sha256:790c42f2a080a7cf54049cdfc027799d8121fa3205bf53e6ace200afddbcdad3 size: 2415


✅ Agent created/updated: arn:aws:bedrock-agentcore:us-west-2:615299776985:runtime/strands_langgraph_agentcore-pJpRgs2mGU
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:✅ Agent created/updated: arn:aws:bedrock-agentcore:us-west-2:615299776985:runtime/strands_langgraph_agentcore-pJpRgs2mGU
Polling for endpoint to be ready...
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Polling for endpoint to be ready...
Agent endpoint: arn:aws:bedrock-agentcore:us-west-2:615299776985:runtime/strands_langgraph_agentcore-pJpRgs2mGU/runtime-endpoint/DEFAULT
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Agent endpoint: arn:aws:bedrock-agentcore:us-west-2:615299776985:runtime/strands_langgraph_agentcore-pJpRgs2mGU/runtime-endpoint/DEFAULT
Deployed to cloud: arn:aws:bedrock-agentcore:us-west-2:615299776985:runtime/strands_langgraph_agentcore-pJpRgs2mGU
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:Deployed to cloud: arn:aws:b

In [14]:
import platform
print(f"Machine: {platform.machine()}")
print(f"Processor: {platform.processor()}")
print(f"Architecture: {platform.architecture()}")
print(f"Platform: {platform.platform()}")

Machine: aarch64
Processor: aarch64
Architecture: ('64bit', 'ELF')
Platform: Linux-6.8.0-1033-aws-aarch64-with-glibc2.35


### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [15]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_langgraph_agentcore
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:Retrieved Bedrock AgentCore status for: strands_langgraph_agentcore


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [17]:
#invoke_response = agentcore_runtime.invoke({"prompt": "How much is 2+2?"})
invoke_response = agentcore_runtime.invoke({"prompt": "안녕 나는 장동진이라고 해. 만나서 반가워 나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어."})
invoke_response

Invoking BedrockAgentCore agent 'strands_langgraph_agentcore' via cloud endpoint
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.invoke:Invoking BedrockAgentCore agent 'strands_langgraph_agentcore' via cloud endpoint


{'ResponseMetadata': {'RequestId': 'b6108f92-5531-4b47-b2c9-edd11c6f8c82',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 06 Aug 2025 04:39:33 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b6108f92-5531-4b47-b2c9-edd11c6f8c82',
   'baggage': 'Self=1-6892dc78-312f890338240029679415a8,session.id=61f23594-847e-4a61-9be1-c4f6304ff621',
   'x-amzn-bedrock-agentcore-runtime-session-id': '61f23594-847e-4a61-9be1-c4f6304ff621',
   'x-amzn-trace-id': 'Root=1-6892dc78-246efc495ca5430d60cbc521;Self=1-6892dc78-312f890338240029679415a8'},
  'RetryAttempts': 0},
 'runtimeSessionId': '61f23594-847e-4a61-9be1-c4f6304ff621',
 'traceId': 'Root=1-6892dc78-246efc495ca5430d60cbc521;Self=1-6892dc78-312f890338240029679415a8',
 'baggage': 'Self=1-6892dc78-312f890338240029679415a8,session.id=61f23594-847e-4a61-9be1-c4f6304ff621',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': [b'{"full_plan"

In [31]:
invoke_response['response'][0].decode("utf-8")

'{"full_plan": "# Plan\\n\\n## thought\\n- 사용자가 데이터 분석과 인사이트 도출을 원하지만, 구체적인 데이터나 분석 방향이 제시되지 않았습니다.\\n- 먼저 사용자의 구체적인 요구사항과 데이터를 파악하는 것이 필요합니다.\\n- 현재는 초기 단계이므로, 기본적인 분석 프레임워크를 제시하고 사용자와의 상호작용을 통해 구체화할 필요가 있습니다.\\n\\n## title\\n데이터 분석 및 인사이트 도출을 위한 기초 프레임워크 수립\\n\\n## steps\\n### 1. Reporter: 초기 요구사항 명확화 및 프레임워크 제안\\n- [ ] 사용자 소개에 대한 응답 및 환영 메시지 작성\\n- [ ] 데이터 분석을 위한 기본적인 정보 요청사항 정리\\n  - 분석하고자 하는 데이터의 종류\\n  - 데이터의 형식 (CSV, Excel, Database 등)\\n  - 분석의 목적과 기대하는 인사이트\\n  - 특별히 관심 있는 분야나 변수\\n- [ ] 데이터 분석 프로세스 개요 설명\\n- '

In [48]:
aa = ''.join(invoke_response['response'].decode("utf-8"))

AttributeError: 'list' object has no attribute 'decode'

In [41]:
invoke_response['response'][0].decode("utf-8")

'{"full_plan": "# Plan\\n## thought\\n사용자는 한국어로 소통하며, 데이터를 제공하고 그로부터 인사이트를 얻고 싶다고 했습니다. 하지만 아직 구체적인 데이터나 분석 요구사항이 제공되지 않았습니다. 따라서 먼저 사용자의 요구사항을 명확히 이해한 후, 데이터 분석 및 인사이트 도출을 위한 계획을 세워야 합니다.\\n## title:\\n데이터 기반 인사이트 도출 프로젝트\\n## steps:\\n### 1. Coder: 데이터 요청 및 초기 분석 준비\\n- [ ] 사용자에게 분석하고자 하는 데이터 유형과 형식 확인\\n- [ ] 데이터 수집 방법 제안 (파일 업로드, API 연결 등)\\n- [ ] 분석 목적과 원하는 인사이트 유형 파악\\n- [ ] 데이터 분석을 위한 초기 코드 프레임워크 준비\\n\\n### 2. Reporter: 최종 보고서 작성\\n- [ ] 수집된 데이터 요약 및 특성 설명\\n- [ ] 데이터 분석 결과 시각화\\n- [ ] 주요 인사이트 및 패턴 도출\\n- [ ] 결론 및'

In [42]:
invoke_response['response'][1].decode("utf-8")

' 추가 분석 제안사항 작성\\n- [ ] 한국어로 최종 보고서 작성", "messages": [{"role": "user", "content": [{"text": "# Plan\\n## thought\\n사용자는 한국어로 소통하며, 데이터를 제공하고 그로부터 인사이트를 얻고 싶다고 했습니다. 하지만 아직 구체적인 데이터나 분석 요구사항이 제공되지 않았습니다. 따라서 먼저 사용자의 요구사항을 명확히 이해한 후, 데이터 분석 및 인사이트 도출을 위한 계획을 세워야 합니다.\\n## title:\\n데이터 기반 인사이트 도출 프로젝트\\n## steps:\\n### 1. Coder: 데이터 요청 및 초기 분석 준비\\n- [ ] 사용자에게 분석하고자 하는 데이터 유형과 형식 확인\\n- [ ] 데이터 수집 방법 제안 (파일 업로드, API 연결 등)\\n- [ ] 분석 목적과 원하는 인사이트 유형 파악\\n- [ ] 데이터 분석을 위한 초기 코드 프레임워크 준비\\n\\n### 2. Reporter: 최종 보고서 작성\\n- [ ] 수집된 데이터 요약 및'

In [43]:
invoke_response['response'][2].decode("utf-8")

' 특성 설명\\n- [ ] 데이터 분석 결과 시각화\\n- [ ] 주요 인사이트 및 패턴 도출\\n- [ ] 결론 및 추가 분석 제안사항 작성\\n- [ ] 한국어로 최종 보고서 작성"}]}], "messages_name": "planner", "history": [{"agent": "coordinator", "message": "안녕하세요 장동진님! 저는 Bedrock-Manus입니다. 만나서 반갑습니다!\\n\\n데이터 분석과 인사이트 추출에 대한 요청은 저희 플래너 시스템과 상담이 필요할 것 같습니다.\\n\\nhandoff_to_planner: I\'ll need to consult our planning system for this request."}, {"agent": "planner", "message": "# Plan\\n## thought\\n사용자는 한국어로 소통하며, 데이터를 제공하고 그로부터 인사이트를 얻고 싶다고 했습니다. 하지만 아직 구체적인 데이터나 분석 요구사항이 제공되지 않았습니다. 따라서 먼저 사용자의 요구사항을 명확히 이해한 후, 데이터 분석 및 인사이트 도출을 위한 계획을 세워야 합니다.\\n## title:\\n'

In [ ]:
invoke_response['response'][3]#.decode("utf-8")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xec in position 1023: unexpected end of data

In [32]:
json.loads(invoke_response['response'][0].decode("utf-8"))

JSONDecodeError: Unterminated string starting at: line 1 column 15 (char 14)

In [30]:
from IPython.display import Markdown, display
import json
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

JSONDecodeError: Unterminated string starting at: line 1 column 15 (char 14)

In [ ]:
스트림

https://strandsagents.com/latest/documentation/docs/user-guide/deploy/deploy_to_bedrock_agentcore/

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is the weather now?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                logger.info(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

### Processing invocation results

We can now process our invocation results to include it in an application

In [ ]:
from IPython.display import Markdown, display
import json
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [28]:
import boto3
import json
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    #payload=json.dumps({"prompt": "What is the weather now?"})
    payload=json.dumps({"prompt": "안녕 나는 장동진이라고 해. 만나서 반가워 나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어."})
    #agentcore_runtime.invoke({"prompt": "안녕 나는 장동진이라고 해. 만나서 반가워 나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어."})
)
# if "text/event-stream" in boto3_response.get("contentType", ""):
#     content = []
#     for line in boto3_response["response"].iter_lines(chunk_size=1):
#         if line:
#             line = line.decode("utf-8")
#             if line.startswith("data: "):
#                 line = line[6:]
#                 logger.info(line)
#                 content.append(line)
#     display(Markdown("\n".join(content)))
# else:
#     try:
#         events = []
#         for event in boto3_response.get("response", []):
#             events.append(event)
#     except Exception as e:
#         events = [f"Error reading EventStream: {e}"]
#     display(Markdown(json.loads(events[0].decode("utf-8"))))

In [29]:
boto3_response

{'ResponseMetadata': {'RequestId': 'd2f783a1-80c5-4728-ac84-a1ed339d173f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 06 Aug 2025 02:03:21 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd2f783a1-80c5-4728-ac84-a1ed339d173f',
   'baggage': 'Self=1-6892b7d7-4c06c892527a66162693c8a4,session.id=ac7fb602-2130-490d-8ca2-aa86de3be3b3',
   'x-amzn-bedrock-agentcore-runtime-session-id': 'ac7fb602-2130-490d-8ca2-aa86de3be3b3',
   'x-amzn-trace-id': 'Root=1-6892b7d7-4705acbd445e51fd08765686;Self=1-6892b7d7-4c06c892527a66162693c8a4'},
  'RetryAttempts': 0},
 'runtimeSessionId': 'ac7fb602-2130-490d-8ca2-aa86de3be3b3',
 'traceId': 'Root=1-6892b7d7-4705acbd445e51fd08765686;Self=1-6892b7d7-4c06c892527a66162693c8a4',
 'baggage': 'Self=1-6892b7d7-4c06c892527a66162693c8a4,session.id=ac7fb602-2130-490d-8ca2-aa86de3be3b3',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': <botocore.respo

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)
policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_iam_role['Role']['RoleName']
)

# Congratulations!